In [1]:
import os
from collections import Counter

import psycopg2

In [2]:
racine_legi = '/home/michel/legi_plat/'
racine_jorf = '/home/michel/jorf_plat/'


In [3]:
categories = [
    'TNC_en_vigueur_JORF',
    'TNC_en_vigueur_LEGI',
    'code_en_vigueur_LEGI',
    'TNC_non_vigueur_JORF',
    'TNC_non_vigueur_LEGI',
    'code_non_vigueur_LEGI',
]

In [4]:
def trouve_enfants(dossier):
    enfants = os.listdir(dossier)
    est_fichier_liste = [os.path.isfile(os.path.join(dossier, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]
    
    return enfants_fichier, enfants_dossier

## Unicité de la catégorie pour un cid

## cid présents dans plusieurs catégories

In [5]:
liste_cid_par_categorie = {}
set_cid_par_categorie = {}
for categorie in categories:
    liste_cid_par_categorie[categorie] = list(os.listdir(os.path.join(racine_legi, categorie)))
    set_cid_par_categorie[categorie] = set(liste_cid_par_categorie[categorie])

In [6]:
nb_cid_par_categorie = {cat:len(l) for cat, l in liste_cid_par_categorie.items()}

In [7]:
nb_cid_par_categorie, sum(nb_cid_par_categorie.values())

({'TNC_en_vigueur_JORF': 76944,
  'TNC_en_vigueur_LEGI': 1671,
  'TNC_non_vigueur_JORF': 17434,
  'TNC_non_vigueur_LEGI': 1771,
  'code_en_vigueur_LEGI': 75,
  'code_non_vigueur_LEGI': 31},
 97926)

In [8]:
liste_cid = [cid for categorie in categories for cid in liste_cid_par_categorie[categorie]]

In [9]:
len(liste_cid)

97926

In [10]:
set_cid = set(liste_cid)

In [11]:
len(set_cid)

97040

In [12]:
print('{:.2f}% des cid apparaissent dans plusieurs catégories'.format((len(liste_cid) - len(set_cid))/len(liste_cid) * 100))

0.90% des cid apparaissent dans plusieurs catégories


In [13]:
categorie_par_cid = {}
liste_cid_unique = list(set_cid)
for cid in liste_cid_unique:
    categorie_par_cid[cid] = [categorie for categorie in categories if cid in set_cid_par_categorie[categorie]]

In [14]:
nb_categorie_par_cid = {cid: len(categorie_par_cid[cid]) for cid in liste_cid_unique}

In [15]:
Counter(nb_categorie_par_cid.values())

Counter({1: 96154, 2: 886})

In [16]:
liste_cid_couple = [cid for cid in liste_cid_unique if len(categorie_par_cid[cid])>1]

In [17]:
for cid in liste_cid_couple:
    print(cid)

JORFTEXT000026973498
JORFTEXT000000751687
JORFTEXT000031872859
JORFTEXT000000271326
JORFTEXT000000625197
JORFTEXT000000591462
JORFTEXT000000803817
JORFTEXT000022337443
JORFTEXT000031593932
JORFTEXT000000225214
JORFTEXT000021530237
JORFTEXT000031319729
JORFTEXT000031319601
JORFTEXT000029350151
JORFTEXT000031664951
JORFTEXT000030920982
JORFTEXT000020021322
JORFTEXT000022972389
JORFTEXT000031601403
JORFTEXT000031400959
JORFTEXT000000591533
JORFTEXT000017981327
JORFTEXT000000555331
JORFTEXT000028745738
JORFTEXT000000603479
JORFTEXT000029796784
JORFTEXT000000497700
JORFTEXT000000550717
JORFTEXT000000584510
JORFTEXT000030707288
JORFTEXT000022363454
JORFTEXT000030001622
JORFTEXT000031602136
JORFTEXT000028353894
JORFTEXT000031680501
JORFTEXT000028745578
JORFTEXT000030785114
JORFTEXT000031585331
JORFTEXT000023375359
JORFTEXT000000603867
JORFTEXT000031585721
JORFTEXT000030967544
JORFTEXT000031644177
JORFTEXT000026365428
JORFTEXT000000652291
JORFTEXT000031665094
JORFTEXT000023474689
JORFTEXT00000

Il y a 886 cid qui apparaissent dans 2 catégories.

In [ ]:
couples_categorie = [categorie_par_cid[cid] for cid in liste_cid_unique if len(categorie_par_cid[cid])>1]

In [ ]:
couples_categorie = [sorted(c) for c in couples_categorie]

In [ ]:
couples_categorie = [(c[0], c[1]) for c in couples_categorie]

In [ ]:
Counter(couples_categorie)

## Comparaison struct et version

In [ ]:
legi_structs_par_cid = {}
legi_versions_par_cid = {}
for categorie in categories:
    _, liste_cid = trouve_enfants((os.path.join(racine_legi, categorie)))

    etat_par_cid = {}
    for cid in liste_cid:
        if cid in liste_cid_couple:
            continue
            
        liste_struct, _ = trouve_enfants(os.path.join(os.path.join(racine_legi, categorie, cid, 'struct')))
        liste_version, _ = trouve_enfants(os.path.join(os.path.join(racine_legi, categorie, cid, 'version')))
        
        legi_structs_par_cid[cid] = [f[:20] for f in liste_struct]
        legi_versions_par_cid[cid] = [f[:20] for f in liste_version]


In [ ]:
legi_structs_par_cid

In [ ]:
cid_present = {cid: cid in liste_id_ for cid, liste_id_ in legi_versions_par_cid.items()}

In [ ]:
liste_cid_present = [cid for cid, est_present in cid_present.items() if est_present]
liste_cid_absent = [cid for cid, est_present in cid_present.items() if not est_present]


In [ ]:
prefixes_present = [cid[:8] for cid in liste_cid_present]
prefixes_absent = [cid[:8] for cid in liste_cid_absent]


In [ ]:
Counter(prefixes_present), Counter(prefixes_absent)

In [ ]:
jorf_liste_struct, _ = trouve_enfants(os.path.join(os.path.join(racine_jorf, 'texte/struct')))
jorf_liste_version, _ = trouve_enfants(os.path.join(os.path.join(racine_jorf, 'texte/version')))

jorf_liste_struct = [f[:20] for f in jorf_liste_struct]
jorf_liste_version = [f[:20] for f in jorf_liste_version]


In [ ]:
jorf_set_struct = set(jorf_liste_struct)
jorf_set_version = set(jorf_liste_version)

In [ ]:
present_dans_jorf = [cid for cid in liste_cid_absent if cid in jorf_set_struct]
absent_dans_jorf = [cid for cid in liste_cid_absent if cid not in jorf_set_struct]


In [ ]:
absent_dans_jorf

In [ ]:
len(liste_cid_absent)

In [ ]:
jorf_liste_struct

In [ ]:
for categorie in categories:
    liste_cid = list(os.listdir(os.path.join(racine_legi, categorie)))

    etat_par_cid = {}
    for cid in liste_cid:
        liste_struct = sorted(list(os.listdir(os.path.join(racine_legi, categorie, cid, 'struct'))))
        liste_version = sorted(list(os.listdir(os.path.join(racine_legi, categorie, cid, 'version'))))
        etat_par_cid[cid] = (liste_struct == liste_version)
        nb_struct_par_cid[cid] = len(liste_struct)
        
    liste_cid_diff = [cid for cid in liste_cid if etat_par_cid[cid] == False]
    
    nb_struct_par_cid = {cid: nb_struct_par_cid[cid] if etat_par_cid[cid] else -1 for cid in liste_cid}
    compteur_taille = dict(Counter(nb_struct_par_cid.values()))
    liste_cid_zero = [cid for cid in liste_cid if nb_struct_par_cid[cid] == 0]
    
    nb_cid = len(liste_cid)
    nb_cid_diff = len(liste_cid_diff)
    nb_cid_zero = len(liste_cid_zero)    
    
    print('Catégorie {} :'.format(categorie))
    print('    {} / {} ont des sous-dossiers différents.'.format(nb_cid_diff, nb_cid))
    print('    -> ' + ', '.join(liste_cid_diff))
    print('    Répartition des tailles de sous-dossiers : {}'.format(str(compteur_taille)))
    print("    {} / {} ont des sous-dossiers vides.".format(nb_cid_zero, nb_cid))
    print('    -> ' + ', '.join(liste_cid_zero))
    print()


## Met en base ces résultats

* Créer la table :
```
create table texte (cid varchar(21), categorie_unique boolean, categorie varchar(21), correspondance_struct_version boolean, struct_version_non_vide boolean, etat varchar(21));
```

* Créer un index :
```
create index on texte (cid);
```

NB : struct_version_non_vide n'est pas forcément un bon motif d'invalidité, car la base JORF contient des fichiers struct et version qui peuvent correspondre.

In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [ ]:
for cid in liste_cid_unique:
    nb_categorie = nb_categorie_par_cid[cid]
    
    if nb_categorie > 1:
        curseur.execute("insert into texte (cid, categorie_unique, etat) values (%s, %s, %s)",
                        (cid, False, 'anomalie'))
        connection.commit()
        
    else:
        categorie = categorie_par_cid[cid][0]
        
        liste_struct = sorted(list(os.listdir(os.path.join(racine_legi, categorie, cid, 'struct'))))
        liste_version = sorted(list(os.listdir(os.path.join(racine_legi, categorie, cid, 'version'))))
        correspondance_struct_version = (liste_struct == liste_version)
        struct_version_non_vide = (len(liste_struct) > 0) if correspondance_struct_version else None

        curseur.execute("""
            insert into texte (
                cid,
                categorie_unique,
                categorie,
                correspondance_struct_version,
                struct_version_non_vide,
                etat
                ) values (%s, %s, %s, %s, %s, %s)
            """, (
                cid,
                True,
                categorie,
                correspondance_struct_version,
                struct_version_non_vide,
                'valide' if (correspondance_struct_version and struct_version_non_vide) else 'anomalie',
            ))
        connection.commit()


In [ ]:
curseur.close()
connection.close()

## Unicité des articles et des section_ta (lorsque categorie_unique)

In [ ]:
liste_fichier = []
liste_fichier_par_cid = {}
for cid in liste_cid_unique:
    if nb_categorie_par_cid[cid] > 1:
        continue
        
    categorie = categorie_par_cid[cid][0]        
    enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_legi, categorie, cid))
    
    liste_fichier_par_cid[cid] = enfants_fichier
    liste_fichier += enfants_fichier

In [ ]:
len(liste_fichier), len(set(liste_fichier))

In [ ]:
Counter(Counter(liste_fichier).values())

In [ ]:
liste_duplication = [article for article, compte in Counter(liste_fichier).items() if compte > 1]

In [ ]:
len(liste_duplication)

In [ ]:
', '.join(liste_duplication)

In [ ]:
set_duplication = set(liste_duplication)

Ajout de la colonne `non_duplication_article_section_ta` :
```
alter table texte add non_duplication_article_section_ta boolean;
```

In [ ]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [ ]:
liste_cid_duplication = []
for cid in liste_cid_unique:
    if (nb_categorie_par_cid[cid] > 1):
        continue
        
    if set(liste_fichier_par_cid[cid]) & set_duplication:
        liste_cid_duplication.append(cid)
        non_duplication_article_section_ta = False
    else:
        non_duplication_article_section_ta = True
        
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    etat = 'valide' if ((etat_vieux == 'valide') and non_duplication_article_section_ta) else 'anomalie'
    
    curseur.execute("update texte set non_duplication_article_section_ta = %s, etat = %s where cid = %s;",
                    (non_duplication_article_section_ta, etat, cid))
    connection.commit()

In [ ]:
len(liste_cid_duplication), ', '.join(liste_cid_duplication)

In [ ]:
curseur.close()
connection.close()

95663 cid valides